#### Nama : Kharisma Gilang P
#### NIM : 175410127
---

# Python untuk Machine Learning

# An introduction to machine learning with scikit-learn

## Loading example dataset

scikit-learn dilengkapi dengan beberapa dataset standar, misalnya set data iris dan digit untuk klasifikasi dan [diabetes dataset](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) untuk regresi.

In [1]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.1)


In [0]:
from sklearn import datasets
iris = datasets.load_iris()
digits = datasets.load_digits()

Dataset adalah objek seperti kamus yang menampung semua data dan beberapa metadata tentang data.

Misalnya, dalam kasus dataset digit, digits.data memberikan akses ke fitur yang dapat digunakan untuk mengklasifikasikan sampel digit:

In [3]:
print(digits.data)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]


dan digits.target memberikan kebenaran dasar untuk dataset digit, yaitu angka yang sesuai dengan setiap gambar digit yang kami coba pelajari:

In [4]:
digits.target

array([0, 1, 2, ..., 8, 9, 8])

## Shape of the data arrays

Data selalu berupa array 2D, shape (n_samples, n_features), meskipun data asli mungkin memiliki bentuk yang berbeda. Dalam hal digit, setiap sampel asli adalah gambar dari shape (8, 8) dan dapat diakses menggunakan:

In [5]:
digits.images[0]

array([[ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.],
       [ 0.,  0., 13., 15., 10., 15.,  5.,  0.],
       [ 0.,  3., 15.,  2.,  0., 11.,  8.,  0.],
       [ 0.,  4., 12.,  0.,  0.,  8.,  8.,  0.],
       [ 0.,  5.,  8.,  0.,  0.,  9.,  8.,  0.],
       [ 0.,  4., 11.,  0.,  1., 12.,  7.,  0.],
       [ 0.,  2., 14.,  5., 10., 12.,  0.,  0.],
       [ 0.,  0.,  6., 13., 10.,  0.,  0.,  0.]])

## Learning and predicting

Dalam kasus dataset digit, tugasnya adalah memprediksi, diberikan gambar, digit mana yang diwakilinya.

Contoh estimator adalah class sklearn.svm.SVC, yang mengimplementasikan [support vector classification](https://en.wikipedia.org/wiki/Support_vector_machine). Konstruktor estimator menggunakan parameter model sebagai argumen.

Untuk saat ini, estimator sebagai black box:

In [0]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

clf model (untuk classifier) estimator instance pertama kali dipasang pada model; model harus belajar untuk melakukan klasifikasi

In [7]:
clf.fit(digits.data[:-1], digits.target[:-1])

SVC(C=100.0, gamma=0.001)

Sekarang kita bisa memprediksi nilai yang baru, kita akan memprediksi menggunakan gambar terakhir dari digits.data.

In [8]:
clf.predict(digits.data[-1:])

array([8])

## Model persistence

Ini memungkinkan untuk menyimpan sebuah model di scikit-learn dengan menggunakan Python built-in persistence model, pickle:

In [9]:
from sklearn import svm
from sklearn import datasets
clf = svm.SVC()
X, y = datasets.load_iris(return_X_y = True)
clf.fit(X, y)

SVC()

In [10]:
import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
clf2.predict(X[0:1])

array([0])

In [11]:
y[0]

0

Dalam kasus tertentu daru scikit-learn, akan lebih menarik jika menggunakan joblib daripada pickle (joblib.dump & joblib.load), dimana lebih efisien untuk data yang besar namun hanya dapat melakukan pickle ke dalam disk dan bukan ke string:

In [12]:
from joblib import dump, load
dump(clf, 'filename.joblib')

['filename.joblib']

Kemudian, kita dapat memuat ulang model pickle dengan:

In [0]:
clf = load('filename.joblib')

> **Note** functions joblib.dump dan joblib.load juga menerima file object daripada filenames.

## Conventions

scikit-learn estimators mengikuti aturan tertentu untuk membuat perilakunya lebih prediktif.

## Type casting

Kecuali jika di spesifikasikan untuk tipe lain, input akan memiliki type `float64`:

In [14]:
import numpy as np
from sklearn import random_projection

rng = np.random.RandomState(0)
X = rng.rand(10, 2000)
X = np.array(X, dtype='float32')
X.dtype

dtype('float32')

In [15]:
transformer = random_projection.GaussianRandomProjection()
X_new = transformer.fit_transform(X)
X_new.dtype

dtype('float64')

Dalam contoh tersebut, `X` adalah `float32`, dimana akan di rubah ke `float64` oleh `fit_transform(X)`.

In [16]:
from sklearn import datasets
from sklearn.svm import SVC
iris = datasets.load_iris()
clf = SVC()
clf.fit(iris.data, iris.target)

SVC()

In [17]:
list(clf.predict(iris.data[:3]))

[0, 0, 0]

In [18]:
clf.fit(iris.data, iris.target_names[iris.target])

SVC()

In [19]:
list(clf.predict(iris.data[:3]))

['setosa', 'setosa', 'setosa']

## Refitting and updating parameters

Hyper-parameters dari estimator dapat di update setelah dibagun melalui metode set_params().

In [0]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
X, y = load_iris(return_X_y=True)

In [21]:
clf = SVC()
clf.set_params(kernel="linear").fit(X, y)

SVC(kernel='linear')

In [22]:
clf.predict(X[:5])

array([0, 0, 0, 0, 0])

In [23]:
clf.set_params(kernel="rbf").fit(X, y)

SVC()

In [24]:
clf.predict(X[:5])

array([0, 0, 0, 0, 0])

## Multiclass vs. multilabel fitting

Ketika menggunakan multiclass classifiers, tugas pembelajaran dan prediksi yang dilakukan tergantung pada format data target yang cocok:

In [25]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

X = [[1, 2], [2, 4], [4, 5], [3, 2], [3, 1]]
y = [0, 0, 1, 1, 2]

classif = OneVsRestClassifier(estimator=SVC(random_state=0))
classif.fit(X, y).predict(X)

array([0, 0, 1, 1, 2])

Dalam kasus di atas, classifier cocok pada array 1d label multiclass dan metode `predict()` memberikan prediksi multiclass yang sesuai. Juga di mungkinkan untuk fit pada 2d array oleh indikator label biner:

In [26]:
y = LabelBinarizer().fit_transform(y)
classif.fit(X, y).predict(X)

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0]])

Di sini, classifier `fit()` pada label biner 2d representasi dari `y`, menggunakan [LabelBinarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html#sklearn.preprocessing.LabelBinarizer). Dalam hal ini `predict()` mengembalikan array 2d yang mewakili prediksi multilabel yang sesuai.

In [27]:
from sklearn.preprocessing import MultiLabelBinarizer
y = [[0, 1], [0, 2], [1, 3], [0, 2, 3], [2, 4]]
y = MultiLabelBinarizer().fit_transform(y)
classif.fit(X, y).predict(X)

array([[1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 1, 0, 0]])

Dalam hal ini, penggolongnya sesuai pada setiap instance yang diberi beberapa label. [MultiLabelBinarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html#sklearn.preprocessing.MultiLabelBinarizer) digunakan untuk membuat binarize array 2d dari multilabel agar sesuai. Hasilnya, `predict()` mengembalikan array 2d dengan beberapa label yang diprediksi untuk setiap instance.